In [1]:
import re
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## Helpers

In [2]:
def get_query_jsn(url):
    """returns json from request
    -INPUT:
        -url: str of url to be requested
    -OUTPUT:
        - list of päevakorrad"""
    return json.loads(requests.get(url).content)

In [3]:
def get_soup(link):
    """returns beautifulsoup object from link content
    -INPUT:
        -link: str of url to be requested
    -OUTPUT:
        - beautifulsoup of link content"""
    soup=BeautifulSoup(requests.get(link).content)
    return soup

In [45]:
#historic faction data is in html
def get_soup_faction_members(soup):
    """function to get historic faction members information
    -INPUT:
        -soup: beautifulsoup ob ject of page where faction member info is
    -OUTPUT:
        -list of faction members information"""
    factions=[]
    faction_name=soup.find('h3').text
    faction_members=[]
    for i, tag in enumerate(soup.find('h3').next_siblings):
        if tag.name=='h3' or tag.name=='h2':
            faction_name=tag.text
            factions.extend(faction_members)
            faction_members=[]
        #esimehed, aseesimehed
        if tag.name=='p':
            text=tag.text
            text=text.replace('Aseesimees: ', '').replace('Esimees: ', '').replace('Aseesimehed: ', '').replace('Esimehed: ', '').replace('\xa0', ' ')
            names=text.split(', ')
            for name in names:
                for subname in name.split('\n'):
                    #usually names not so long
                    if len(subname.split())<10 and subname!='':
                        faction_members.append((subname, faction_name))
        #list of faction members
        if tag.name=='ul' or tag.name=='ol':
            list_members=[(li.text, faction_name) for li in tag.findAll('li')]
            faction_members.extend(list_members)

    factions.extend(faction_members)
    return factions

In [46]:
def get_koosseis_faction_members(koosseisud=['xi', 'xii', 'xiii', 'xiv']):
    """get faction members from Riigikogu koosseisud
    -INPUT:
        -koosseisud:  list of str of koosseis number
    -OUTPIT:
        -pd.DataFrame of faction members information"""
    df_factions=pd.DataFrame()
    for koosseis in koosseisud:
        print(f'working in faction {koosseis}')
        soup=get_soup(f'https://www.riigikogu.ee/tutvustus-ja-ajalugu/riigikogu-ajalugu/{koosseis}-riigikogu-koosseis/fraktsioonid/')
        faction_members=get_soup_faction_members(soup)
        df_=pd.DataFrame(faction_members)
        print(f'about {df_.shape[0]} members on {koosseis}')
        df_.columns=['name', 'faction']
        df_['koosseis']=koosseis
        df_factions=df_factions.append(df_)
    df_factions=df_factions.drop_duplicates()
    df_factions=df_factions.reset_index(drop=True)
    return df_factions

## Get all historic factions from html pages

In [47]:
df_factions=get_koosseis_faction_members()

working in faction xi
about 108 members on xi
working in faction xii
about 101 members on xii
working in faction xiii


C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_11652\2158127128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_factions=df_factions.append(df_)
C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_11652\2158127128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_factions=df_factions.append(df_)
C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_11652\2158127128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_factions=df_factions.append(df_)


about 110 members on xiii
working in faction xiv
about 106 members on xiv


C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_11652\2158127128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_factions=df_factions.append(df_)


In [48]:
df_factions.shape

(411, 3)

In [49]:
df_factions.faction.value_counts()

Eesti Reformierakonna fraktsioon                    124
Eesti Keskerakonna fraktsioon                        96
Sotsiaaldemokraatliku Erakonna fraktsioon            52
Isamaa ja Res Publica Liidu fraktsioon               41
Eesti Konservatiivse Rahvaerakonna fraktsioon        27
Isamaa fraktsioon                                    23
Fraktsioonidesse mittekuulunud Riigikogu liikmed     16
Fraktsiooni mittekuuluvad saadikud                    8
Eestimaa Rahvaliidu fraktsioon                        7
Erakonna Eestimaa Rohelised fraktsioon                6
Eesti Vabaerakonna fraktsioon                         6
Fraktsiooni mittekuuluvad saadikud                    5
Name: faction, dtype: int64

In [50]:
df_factions.head()

,name,faction,koosseis
0,Mai Treial (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi
1,Villu Reiljan (02.04.2007–26.05.2010),Eestimaa Rahvaliidu fraktsioon,xi
2,Karel Rüütli (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi
3,Ester Tuiksoo (09.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi
4,Tarmo Mänd (03.04.2007–12.03.2010),Eestimaa Rahvaliidu fraktsioon,xi


## Get current koosseis using API

In [51]:
current_koosseis=get_query_jsn('https://api.riigikogu.ee/api/plenary-members?lang=et')

In [52]:
df_current_faction=pd.DataFrame([[i['fullName'], i['factions'][0]['name']] for i in current_koosseis])
df_current_faction.columns=['name', 'faction']
df_current_faction['koosseis']='xv'
df_current_faction.head()

,name,faction,koosseis
0,Jaak Aab,Fraktsiooni mittekuuluvad Riigikogu liikmed,xv
1,Annely Akkermann,Eesti Reformierakonna fraktsioon,xv
2,Yoko Alender,Eesti Reformierakonna fraktsioon,xv
3,Anti Allas,Sotsiaaldemokraatliku Erakonna fraktsioon,xv
4,Arvo Aller,Fraktsiooni mittekuuluvad Riigikogu liikmed,xv


## Append current and historic data

In [53]:
df_factions_all=df_factions.append(df_current_faction)
df_factions_all=df_factions_all.reset_index(drop=True)
df_factions_all.shape

C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_11652\812805853.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_factions_all=df_factions.append(df_current_faction)


(512, 3)

## Do some cleaning

In [54]:
df_factions_all['name'].nunique(), df_factions_all['faction'].nunique()

(325, 14)

In [55]:
#a bit of cleaning and formatting
df_factions_all['name']=df_factions_all.name.str.strip()
df_factions_all['faction']=df_factions_all.faction.str.strip()
df_factions_all['name'].nunique(), df_factions_all['faction'].nunique()

(324, 13)

## Get times (if from or to exists) if person has changed faction during koosseis

In [56]:
def get_from_to(text):
    """extract from and to in faction member nae field"""
    if ' (alates' in text.lower():
        return text.split(' (alates')[-1].replace(')', '').strip(), None
    start_end=re.findall('\(\d\d\.\d\d\.\d\d\d\d–\d\d\.\d\d\.\d\d\d\d\)', text)
    if len(start_end)>0:
        start, end = start_end[0].split('–')
        return start.replace('(', '').strip(), end.replace(')', '').strip()
    return None, None

In [57]:
df_factions_all['from'], df_factions_all['to']= zip(*df_factions_all['name'].map(get_from_to))

In [58]:
df_factions_all['from'].value_counts()

07.07.2010    5
02.04.2007    4
09.04.2007    1
03.04.2007    1
31.05.2010    1
22.04.2008    1
12.03.2010    1
07.12.2010    1
Name: from, dtype: int64

In [59]:
df_factions_all[df_factions_all.name.str.contains('kaja')]

,name,faction,koosseis,from,to
282,Viktoria Ladõnskaja-Kubits,Isamaa fraktsioon,xiii,None,None
394,Viktoria Ladõnskaja-Kubits,Isamaa fraktsioon,xiv,None,None


## Fill members from and to based on koosseis start time

In [60]:
#this was taken from wikipedia
#NB! default time format MM.DD:YYYY
koosseis_start={'xi':'02.04.2007',
                'xii':'04.04.2011',
                'xiii':'30.03.2015',
                'xiv':'04.04.2019',
               'xv':'10.04.2023'}

koosseis_end={'xi':'03.04.2011',
                'xii':'29.03.2015',
                'xiii':'03.04.2019',
                'xiv':'09.04.2023',
              'xv':'09.04.2027'}

koosseis_start={'xi':'04.02.2007',
                'xii':'04.04.2011',
                'xiii':'03.30.2015',
                'xiv':'04.04.2019',
               'xv':'04.10.2023'}

koosseis_end={'xi':'04.03.2011',
                'xii':'03.29.2015',
                'xiii':'04.03.2019',
                'xiv':'04.09.2023',
              'xv':'04.09.2027'}

In [61]:
df_factions_all['from']=np.where(pd.isnull(df_factions_all['from']), df_factions_all.koosseis.apply(lambda x: koosseis_start[x]), df_factions_all['from'])
df_factions_all['to']=np.where(pd.isnull(df_factions_all['to']), df_factions_all.koosseis.apply(lambda x: koosseis_end[x]), df_factions_all['to'])
df_factions_all['from']=pd.to_datetime(df_factions_all['from'])
df_factions_all['to']=pd.to_datetime(df_factions_all['to'])

C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_11652\850594200.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_factions_all['from']=pd.to_datetime(df_factions_all['from'])
C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_11652\850594200.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_factions_all['to']=pd.to_datetime(df_factions_all['to'])


In [62]:
df_factions_all

,name,faction,koosseis,from,to
0,Mai Treial (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-07-07
1,Villu Reiljan (02.04.2007–26.05.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-05-26
2,Karel Rüütli (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-07-07
3,Ester Tuiksoo (09.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-09-04,2010-07-07
4,Tarmo Mänd (03.04.2007–12.03.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-03-04,2010-12-03
...,...,...,...,...,...
507,Aleksandr Tšaplõgin,Fraktsiooni mittekuuluvad Riigikogu liikmed,xv,2023-04-10,2027-04-09
508,Toomas Uibo,Eesti 200 fraktsioon,xv,2023-04-10,2027-04-09
509,Kristo Enn Vaga,Fraktsiooni mittekuuluvad Riigikogu liikmed,xv,2023-04-10,2027-04-09
510,Jaak Valge,Fraktsiooni mittekuuluvad Riigikogu liikmed,xv,2023-04-10,2027-04-09


## Do additional cleaning of names

In [63]:
#remove time info from names
df_factions_all['name_clean']=df_factions_all.name.apply(lambda x: x.split('(')[0].split(', ')[0].strip())
df_factions_all.head()

,name,faction,koosseis,from,to,name_clean
0,Mai Treial (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-07-07,Mai Treial
1,Villu Reiljan (02.04.2007–26.05.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-05-26,Villu Reiljan
2,Karel Rüütli (02.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-02-04,2010-07-07,Karel Rüütli
3,Ester Tuiksoo (09.04.2007–07.07.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-09-04,2010-07-07,Ester Tuiksoo
4,Tarmo Mänd (03.04.2007–12.03.2010),Eestimaa Rahvaliidu fraktsioon,xi,2007-03-04,2010-12-03,Tarmo Mänd


In [64]:
df_factions_all.name_clean.value_counts()

Heljo Pikhof       5
Marko Mihkelson    5
Liikmed            5
Enn Eesmaa         5
Mati Raidma        4
                  ..
Innar Mäesalu      1
Laine Randjärv     1
Aivar Rosenberg    1
Andre Sepp         1
Varro Vooglaid     1
Name: name_clean, Length: 314, dtype: int64

In [65]:
#check if have some funny characters
set(''.join(df_factions_all.name_clean.tolist()).lower())

{' ',
 '-',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 'ä',
 'õ',
 'ö',
 'ü',
 'š'}

In [66]:
df_factions_all[df_factions_all.name_clean.str.contains('-')]

,name,faction,koosseis,from,to,name_clean
18,Mari-Ann Kelam,Isamaa ja Res Publica Liidu fraktsioon,xi,2007-04-02,2011-04-03,Mari-Ann Kelam
24,Liisa-Ly Pakosta,Isamaa ja Res Publica Liidu fraktsioon,xi,2007-04-02,2011-04-03,Liisa-Ly Pakosta
29,Ken-Marti Vaher,Isamaa ja Res Publica Liidu fraktsioon,xi,2007-04-02,2011-04-03,Ken-Marti Vaher
51,Katrin Karisma-Krumm,Eesti Reformierakonna fraktsioon,xi,2007-04-02,2011-04-03,Katrin Karisma-Krumm
65,Paul-Eerik Rummo,Eesti Reformierakonna fraktsioon,xi,2007-04-02,2011-04-03,Paul-Eerik Rummo
119,Liisa-Ly Pakosta,Isamaa ja Res Publica Liidu fraktsioon,xii,2011-04-04,2015-03-29,Liisa-Ly Pakosta
124,Helir-Valdor Seeder,Isamaa ja Res Publica Liidu fraktsioon,xii,2011-04-04,2015-03-29,Helir-Valdor Seeder
129,Ken-Marti Vaher,Isamaa ja Res Publica Liidu fraktsioon,xii,2011-04-04,2015-03-29,Ken-Marti Vaher
167,Tiina Lokk-Tramberg,Eesti Reformierakonna fraktsioon,xii,2011-04-04,2015-03-29,Tiina Lokk-Tramberg
177,Paul-Eerik Rummo,Eesti Reformierakonna fraktsioon,xii,2011-04-04,2015-03-29,Paul-Eerik Rummo


In [67]:
# members that have multiple factions in one koosseis - became MPs without faction (left faction/party)
df_factions_all[df_factions_all.duplicated(subset=['name_clean', 'koosseis'])]

,name,faction,koosseis,from,to,name_clean
101,Tarmo Mänd (alates 12.03.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-12-03,2011-04-03,Tarmo Mänd
102,Jaanus Marrandi (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-04-03,Jaanus Marrandi
103,Karel Rüütli (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-04-03,Karel Rüütli
105,Mai Treial (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-04-03,Mai Treial
106,Ester Tuiksoo (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-04-03,Ester Tuiksoo
107,Jaan Õunapuu (alates 07.07.2010),Fraktsioonidesse mittekuulunud Riigikogu liikmed,xi,2010-07-07,2011-04-03,Jaan Õunapuu
332,Liikmed,Eesti Konservatiivse Rahvaerakonna fraktsioon,xiv,2019-04-04,2023-04-09,Liikmed
352,Liikmed,Eesti Reformierakonna fraktsioon,xiv,2019-04-04,2023-04-09,Liikmed
386,Liikmed,Isamaa fraktsioon,xiv,2019-04-04,2023-04-09,Liikmed
398,Liikmed,Sotsiaaldemokraatliku Erakonna fraktsioon,xiv,2019-04-04,2023-04-09,Liikmed


## Save faction data

In [68]:
df_factions_all.to_csv('data/interim/factions2022.csv',index=False)

In [69]:
df_factions_all.to_excel('data/interim/factions2022.xlsx',index=False)

## NB! Better to use manually reviewed data, there are some mistakes removed

In [92]:
df_factions_all=pd.read_excel('data/interim/factions2022.xlsx')
df_factions_all.shape

(588, 6)

In [93]:
df_factions_all=df_factions_all.drop_duplicates()
df_factions_all.shape

(588, 6)

## Read stenos to add faction info

In [94]:
df_stenos=pd.read_csv('data/interim/stenos_cleaned.csv')
df_stenos.shape

(241366, 14)

In [95]:
df_stenos.head()

,heading,speaker,link_video,link_steno,index_pk,index_snd,text,year,month,day,time,seconds_from_start,video_link_wo_time,text_wo_punct
0,10:00 Istungi rakendamine,Aseesimees Siim Kallas,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=0,https://stenogrammid.riigikogu.ee/201911191000,PKP-25040,SND-559399,"Auväärt kolleegid, auväärt Riigikogu, tere hom...",2019,11,19,10:00,0,https://www.youtube.com/watch?v=GaFL2Zc4OMg,auväärt kolleegid auväärt riigikogu tere hommi...
1,1.\n 10:02 Eesti Keske...,Aseesimees Siim Kallas,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=125,https://stenogrammid.riigikogu.ee/201911191000,PKP-25041,SND-559400,Täna on meil päevakorras olulise tähtsusega ri...,2019,11,19,10:02,125,https://www.youtube.com/watch?v=GaFL2Zc4OMg,täna on meil päevakorras olulise tähtsusega ri...
2,1.\n 10:02 Eesti Keske...,Kersti Sarapuu,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=238,https://stenogrammid.riigikogu.ee/201911191000,PKP-25041,SND-559401,Lugupeetud istungi juhataja! Head kolleegid ja...,2019,11,19,10:02,238,https://www.youtube.com/watch?v=GaFL2Zc4OMg,lugupeetud istungi juhataja head kolleegid ja ...
3,1.\n 10:02 Eesti Keske...,Aseesimees Siim Kallas,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=...,https://stenogrammid.riigikogu.ee/201911191000,PKP-25041,SND-559402,"Aitäh! Ettekandjatele ütlen niipalju, et selle...",2019,11,19,10:02,1194,https://www.youtube.com/watch?v=GaFL2Zc4OMg,aitäh ettekandjatele ütlen niipalju et selles ...
4,1.\n 10:02 Eesti Keske...,Aivar Sõerd,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=...,https://stenogrammid.riigikogu.ee/201911191000,PKP-25041,SND-559403,"Aitäh, austatud juhataja! Austatud ettekandja!...",2019,11,19,10:02,1231,https://www.youtube.com/watch?v=GaFL2Zc4OMg,aitäh austatud juhataja austatud ettekandja ei...


## Add koosseis info

In [96]:
#create koossseis variable
df_stenos['date']=pd.to_datetime(df_stenos.year.astype(str)+'.'+df_stenos.month.astype(str)+'.'+df_stenos.day.astype(str))
df_stenos['koosseis']=np.where(df_stenos['date']<=pd.to_datetime(koosseis_end['xi']), 'xi', '')
df_stenos['koosseis']=np.where((df_stenos['date']<=pd.to_datetime(koosseis_end['xii']))&(df_stenos['date']>pd.to_datetime(koosseis_end['xi'])), 'xii', df_stenos['koosseis'])
df_stenos['koosseis']=np.where((df_stenos['date']<=pd.to_datetime(koosseis_end['xiii']))&(df_stenos['date']>pd.to_datetime(koosseis_end['xii'])), 'xiii', df_stenos['koosseis'])
df_stenos['koosseis']=np.where((df_stenos['date']<=pd.to_datetime(koosseis_end['xiv']))&(df_stenos['date']>pd.to_datetime(koosseis_end['xiii'])), 'xiv', df_stenos['koosseis'])
df_stenos['koosseis']=np.where((df_stenos['date']<=pd.to_datetime(koosseis_end['xv']))&(df_stenos['date']>pd.to_datetime(koosseis_end['xiv'])), 'xv', df_stenos['koosseis'])

In [97]:
pd.to_datetime('04.10.2023').day

10

In [98]:
df_stenos.koosseis.value_counts()

xiv     81251
xiii    72124
xii     57013
xi      19818
xv      11160
Name: koosseis, dtype: int64

In [99]:
df_stenos.shape

(241366, 16)

In [100]:
df_stenos[df_stenos.year==2023].head(10)

,heading,speaker,link_video,link_steno,index_pk,index_snd,text,year,month,day,time,seconds_from_start,video_link_wo_time,text_wo_punct,date,koosseis
197993,13:58 Istungi rakendamine,Aseesimees Toomas Kivimägi,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=168,https://stenogrammid.riigikogu.ee/202304191400,PKP-608089,SND-20688824,"Kohal on 83 Riigikogu liiget, puudub 18.",2023,4,19,14:03,168,https://www.youtube.com/watch?v=CZRDiKBPPnI,kohal on 83 riigikogu liiget puudub 18,2023-04-19,xv
197994,1.\n 14:03 Õiguskantsle...,Aseesimees Toomas Kivimägi,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=205,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688849,Läheme tänase istungi päevakorrapunkti juurde...,2023,4,19,14:03,205,https://www.youtube.com/watch?v=CZRDiKBPPnI,läheme tänase istungi päevakorrapunkti juurde ...,2023-04-19,xv
197995,1.\n 14:03 Õiguskantsle...,Õiguskantsler Ülle Madise,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=267,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688845,Austatud istungi juhataja! Austatud Riigikogu...,2023,4,19,14:04,267,https://www.youtube.com/watch?v=CZRDiKBPPnI,austatud istungi juhataja austatud riigikogu l...,2023-04-19,xv
197996,1.\n 14:03 Õiguskantsle...,Aseesimees Toomas Kivimägi,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=...,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688916,"Väga suur tänu, lugupeetud õiguskantsler! Ava...",2023,4,19,14:24,1472,https://www.youtube.com/watch?v=CZRDiKBPPnI,väga suur tänu lugupeetud õiguskantsler avalda...,2023-04-19,xv
197997,1.\n 14:03 Õiguskantsle...,Anti Poolamets,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=...,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688864,"Aitäh, juhataja! Lugupeetud ettekandja! Tegu ...",2023,4,19,14:25,1480,https://www.youtube.com/watch?v=CZRDiKBPPnI,aitäh juhataja lugupeetud ettekandja tegu on j...,2023-04-19,xv
197998,1.\n 14:03 Õiguskantsle...,Õiguskantsler Ülle Madise,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=...,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688915,Tänan! Teie seisukohta on alati väga huvitav ...,2023,4,19,14:26,1548,https://www.youtube.com/watch?v=CZRDiKBPPnI,tänan teie seisukohta on alati väga huvitav jä...,2023-04-19,xv
197999,1.\n 14:03 Õiguskantsle...,Aseesimees Toomas Kivimägi,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=...,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688893,"Kalle Grünthal, palun!",2023,4,19,14:26,1577,https://www.youtube.com/watch?v=CZRDiKBPPnI,kalle grünthal palun,2023-04-19,xv
198000,1.\n 14:03 Õiguskantsle...,Kalle Grünthal,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=...,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688891,"Aitäh, lugupeetud istungi juhataja! Austatud ...",2023,4,19,14:26,1578,https://www.youtube.com/watch?v=CZRDiKBPPnI,aitäh lugupeetud istungi juhataja austatud ett...,2023-04-19,xv
198001,1.\n 14:03 Õiguskantsle...,Õiguskantsler Ülle Madise,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=...,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688928,"Suur tänu! See, et täna on Riigikogu saal rah...",2023,4,19,14:27,1641,https://www.youtube.com/watch?v=CZRDiKBPPnI,suur tänu see et täna on riigikogu saal rahvae...,2023-04-19,xv
198002,1.\n 14:03 Õiguskantsle...,Aseesimees Toomas Kivimägi,https://www.youtube.com/watch?v=CZRDiKBPPnI&t=...,https://stenogrammid.riigikogu.ee/202304191400,PKP-608090,SND-20688853,"Urve Tiidus, palun!",2023,4,19,14:29,1739,https://www.youtube.com/watch?v=CZRDiKBPPnI,urve tiidus palun,2023-04-19,xv


## Add faction info

In [101]:
#this one takes time!
def get_stenos_factions(x):
    """adds faction member info to steno based on the time of Riigikogu koosseis and when somebody was in faction"""
    df_  = df_stenos[df_stenos.index_snd==x['index_snd']]
    if not df_['speaker'].tolist()[0] in df_factions_all.name_clean.tolist():
         return ''
    df_factions_all_sub=df_factions_all[(df_factions_all['to']>=df_.date.tolist()[0])&(df_factions_all['from']<=df_.date.tolist()[0])&(df_factions_all.name_clean==df_['speaker'].tolist()[0])]
    if df_factions_all_sub.shape[0]==0:
        return ''
    elif df_factions_all_sub.shape[0]==1:
        return df_factions_all_sub['faction'].tolist()[0]
    else:
        print(f'df shape {df_.shape} should be (0, n)')
        
%time stenos_factions=df_stenos.apply(get_stenos_factions, axis=1)

CPU times: total: 23min 25s
Wall time: 41min 25s


In [102]:
df_stenos['fation']=stenos_factions

In [103]:
df_stenos['fation'].value_counts()

                                                    144154
Eesti Reformierakonna fraktsioon                     21621
Eesti Keskerakonna fraktsioon                        21016
Eesti Konservatiivse Rahvaerakonna fraktsioon        16380
Sotsiaaldemokraatliku Erakonna fraktsioon            12443
Isamaa fraktsioon                                     8520
Isamaa ja Res Publica Liidu fraktsioon                5208
Eesti Vabaerakonna fraktsioon                         5200
Fraktsiooni mittekuuluvad saadikud                    3368
Fraktsioonidesse mittekuulunud Riigikogu liikmed      2446
Erakonna Eestimaa Rohelised fraktsioon                 486
Eestimaa Rahvaliidu fraktsioon                         411
Eesti 200 fraktsioon                                   113
Name: fation, dtype: int64

## Save stenos

In [104]:
df_stenos.to_csv('data/interim/stenos_cleaned_faction2022.csv', index=False)